In [15]:
import numpy as np, matplotlib.pyplot as plt
import itertools

from qulacs import QuantumState, QuantumCircuit
from qulacs.gate import Measurement, H, CNOT, CZ, X

from tool.check_encoding import print_state, run_stabilizer_circuit, check_encoding, get_state_dict

# def print_state(state, computational_states, num_data=7):
#     for i, s in enumerate(computational_states):
#         s = s[:num_data] + '|' + s[num_data:]
#         if abs(state[i]) > 1e-6:
#             print(f'{s}: {state[i].real:.3f}  +  {state[i].imag:.3f}i')  

# gatedict = {
#     'H': H,
#     'CX': CNOT,
#     'CZ': CZ,
#     'Meas': Measurement,
# }

# def run_stabilizer_circuit(gate_sequence, num_qubits, num_meas, 
#                            target_outcomes=None, verbose=False):

#     computational_states = [''.join(map(str,i))[::-1] for i in itertools.product([0, 1], repeat=num_qubits)]
#     anc_meas = ''

#     while anc_meas != target_outcomes:

#         state = QuantumState(num_qubits)
#         state.set_zero_state()

#         circuit = QuantumCircuit(num_qubits)
#         for gate, pos in gate_sequence:
#             circuit.add_gate(gatedict[gate](*pos))
#             # reset qubit manually at each Measurement
#             if gate == 'Meas':
#                 circuit.update_quantum_state(state)
#                 if state.get_classical_value(pos[1]) == 1:
#                     circuit = QuantumCircuit(num_qubits)
#                     circuit.add_X_gate(pos[0])
#                     circuit.update_quantum_state(state)
#                 circuit = QuantumCircuit(num_qubits)
#         # run the remaining circuit after the last Measurement
#         if gate != 'Meas':
#             circuit.update_quantum_state(state)   

#         anc_meas = ''.join([str(state.get_classical_value(i)) for i in range(num_meas)])

#         if target_outcomes is None:
#             break

#     if verbose:
#         print(f'ancilla meas: {anc_meas}')
#         print_state(state.get_vector(), computational_states)

#     return state, anc_meas
    
# def get_state_dict(state, computational_states, num_data=7):
#     state_dict = {}
#     for i, s in enumerate(computational_states):
#         s = s[:num_data]
#         if abs(state[i]) > 1e-6:
#             state_dict[s] = state[i]
#     return state_dict

# def check_encoding(gate_sequence, num_qubits, num_meas, lut, correct):
#     computational_states = [''.join(map(str,i))[::-1] for i in itertools.product([0, 1], repeat=num_qubits)]

#     # get code word
#     state, anc_meas = run_stabilizer_circuit(gate_sequence, num_qubits, num_meas, target_outcomes='000')
#     codeword = get_state_dict(state.get_vector(),computational_states)

#     synd_list = list(lut.keys())

#     print('Comparision to logical state before and after correction')
#     afters = []
#     while len(synd_list) > 0:
#         state, anc_meas = run_stabilizer_circuit(gate_sequence, num_qubits, num_meas)
#         before = get_state_dict(state.get_vector(),computational_states) == codeword
#         # print(anc_meas,synd_list)
#         if anc_meas in synd_list:
#             synd_list.remove(anc_meas)
#         else:
#             continue

#         # correct single qubit
#         if anc_meas != '000':
#             circuit = QuantumCircuit(num_qubits)
#             if correct == 'Z':
#                 circuit.add_Z_gate(lut[anc_meas])
#             elif correct == 'X':
#                 circuit.add_X_gate(lut[anc_meas])
#             circuit.update_quantum_state(state)
#         after = get_state_dict(state.get_vector(),computational_states) == codeword
#         afters.append(after)

#         print(f'   synd = {anc_meas}:  {before}  ->  {after}')
#     return all(afters)

# 3-qubit Bell state

In [2]:
n = 4
computational_states = [''.join(map(str,i))[::-1] for i in itertools.product([0, 1], repeat=n)]
num_meas = 2

state = QuantumState(n)
state.set_zero_state()

classical_ind = 0
circuit = QuantumCircuit(n)

circuit.add_H_gate(3)
circuit.add_gate(CNOT(3,0))
circuit.add_gate(CNOT(3,1))
circuit.add_H_gate(3)
circuit.add_gate(Measurement(3,classical_ind))
classical_ind += 1

circuit.add_H_gate(3)
circuit.add_gate(CNOT(3,1))
circuit.add_gate(CNOT(3,2))
circuit.add_H_gate(3)
circuit.add_gate(Measurement(3,classical_ind))
classical_ind += 1

# Operate quantum circuit to state
circuit.update_quantum_state(state)

print([state.get_classical_value(i) for i in range(num_meas)])
print_state(state.get_vector(), computational_states, 3)

[0, 0]
000|0: 0.500  +  0.000i
110|0: 0.500  +  0.000i
101|0: 0.500  +  0.000i
011|0: 0.500  +  0.000i


In [3]:
n = 3
computational_states = [''.join(map(str,i))[::-1] for i in itertools.product([0, 1], repeat=n)]

state = QuantumState(n)
state.set_zero_state()

circuit = QuantumCircuit(n)

# Add hadamard gate to quantum circuit
circuit.add_H_gate(0)

# Create gate, which can also be added
circuit.add_gate(CNOT(0,1))
circuit.add_gate(CNOT(0,2))
circuit.add_gate(Measurement(2,0))
circuit.add_gate(X(2))

# Operate quantum circuit to state
circuit.update_quantum_state(state)

print(state.get_classical_value(0))
print_state(state.get_vector(), computational_states)

0
001|: 1.000  +  0.000i


# Steane code
## No-flag standard circuit with CX

In [4]:
# standard X-checks: X3456, X1256, X0246
gate_sequence = [
    ('H', (7,)),
    *[('CX', (control, target)) for control, target in zip([7]*4,[3,4,5,6])],
    ('H', (7,)), 
    ('Meas', (7,0)),

    ('H', (7,)),
    *[('CX', (control, target)) for control, target in zip([7]*4,[1,2,5,6])],
    ('H', (7,)), 
    ('Meas', (7,1)),

    ('H', (7,)),
    *[('CX', (control, target)) for control, target in zip([7]*4,[0,2,4,6])],
    ('H', (7,)), 
    ('Meas', (7,2)),
]
lut = {
    '000': 1000, '100': 3, '010': 1, '001': 0,
    '110': 5, '101': 4, '011': 2, '111': 6,
}

num_qubits = 8
num_meas = 3

check_encoding(gate_sequence, num_qubits, num_meas, lut, 
               'Z', '\n--- standard X-checks -> logical0 state')


--- standard X-checks -> logical0 state
Comparison to logical state before and after correction
   synd = 100:  False  ->  True
   synd = 110:  False  ->  True
   synd = 001:  False  ->  True
   synd = 000:  True  ->  True
   synd = 011:  False  ->  True
   synd = 101:  False  ->  True
   synd = 010:  False  ->  True
   synd = 111:  False  ->  True


True

In [5]:
# standard Z-checks: Z3456, Z1256, Z0246
gate_sequence = [
    *[('H', (i,)) for i in range(7)],
    *[('CX', (control, target)) for control, target in zip([3,4,5,6],[7]*4)],
    ('Meas', (7,0)),

    *[('CX', (control, target)) for control, target in zip([1,2,5,6],[7]*4)],
    ('Meas', (7,1)),

    *[('CX', (control, target)) for control, target in zip([0,2,4,6],[7]*4)],
    ('Meas', (7,2)),

    # *[('H', (i,)) for i in range(7)],
]
lut = {
    '000': 1000, '100': 3, '010': 1, '001': 0,
    '110': 5, '101': 4, '011': 2, '111': 6,
}

num_qubits = 8
num_meas = 3

check_encoding(gate_sequence, num_qubits, num_meas, lut, 
               'X', '\n--- standard Z-checks -> logical+ state')
check_encoding(gate_sequence + [('H', (i,)) for i in range(7)], num_qubits, num_meas, lut, 
               'Z', '\n--- standard Z-checks + Hadamard -> logical0 state')


--- standard Z-checks -> logical+ state
Comparison to logical state before and after correction
   synd = 010:  False  ->  True
   synd = 110:  False  ->  True
   synd = 001:  False  ->  True
   synd = 000:  True  ->  True
   synd = 011:  False  ->  True
   synd = 111:  False  ->  True
   synd = 100:  False  ->  True
   synd = 101:  False  ->  True

--- standard Z-checks + Hadamard -> logical0 state
Comparison to logical state before and after correction
   synd = 101:  False  ->  True
   synd = 110:  False  ->  True
   synd = 010:  False  ->  True
   synd = 011:  False  ->  True
   synd = 000:  True  ->  True
   synd = 111:  False  ->  True
   synd = 001:  False  ->  True
   synd = 100:  False  ->  True


True

## 2-flag ancilla-centered Z-checks with CX

In [18]:
# Z-checks: Z0123, Z2415, Z5623
gate_sequence = [
    *[('H', (i,)) for i in range(7)],

    ('H', (8,)), ('H', (9,)),
    *[('CX', (control, target)) for control, target in zip([8,9,0,1,2,3,9,8],[7,8,7,7,8,9,8,7])],
    ('H', (8,)), ('H', (9,)),
    ('Meas', (7,0)),

    ('H', (7,)), ('H', (10,)),
    *[('CX', (control, target)) for control, target in zip([7,10,2,4,1,5,10,7],[8,7,8,8,7,10,7,8])],
    ('H', (7,)), ('H', (10,)),
    ('Meas', (8,1)),

    ('H', (8,)), ('H', (9,)),
    *[('CX', (control, target)) for control, target in zip([8,9,5,6,2,3,9,8],[10,8,10,10,8,9,8,10])],
    ('H', (8,)), ('H', (9,)),
    ('Meas', (10,2)),

    # *[('H', (i,)) for i in range(7)],
]
lut = {
    '000': 1000, '100': 0, '010': 4, '001': 6,
    '110': 1, '101': 3, '011': 5, '111': 2,
}

num_qubits = 11
num_meas = 3


check_encoding(gate_sequence, num_qubits, num_meas, lut, 
               'X', '\n--- 2-flag ancilla-centered Z-checks with CX -> logical+ state')
check_encoding(gate_sequence + [('H', (i,)) for i in range(7)], num_qubits, num_meas, lut, 
               'Z', '\n--- 2-flag ancilla-centered Z-checks with CX +  Hadamard -> logical0 state')


--- 2-flag ancilla-centered Z-checks with CX -> logical+ state
Comparison to logical state before and after correction
   synd = 110:  False  ->  True
   synd = 100:  False  ->  True
   synd = 011:  False  ->  True
   synd = 000:  True  ->  True
   synd = 101:  False  ->  True
   synd = 001:  False  ->  True
   synd = 111:  False  ->  True
   synd = 010:  False  ->  True

--- 2-flag ancilla-centered Z-checks with CX +  Hadamard -> logical0 state
Comparison to logical state before and after correction
   synd = 010:  False  ->  True
   synd = 110:  False  ->  True
   synd = 000:  True  ->  True
   synd = 001:  False  ->  True
   synd = 011:  False  ->  True
   synd = 111:  False  ->  True
   synd = 100:  False  ->  True
   synd = 101:  False  ->  True


True

In [9]:
run_stabilizer_circuit(gate_sequence, num_qubits, num_meas, target_outcomes='000', verbose=True)

ancilla meas: 000
0000000|0000: 0.250  +  0.000i
1111000|0000: 0.250  +  0.000i
1100100|0000: 0.250  +  0.000i
0011100|0000: 0.250  +  0.000i
1010010|0000: 0.250  +  0.000i
0101010|0000: 0.250  +  0.000i
0110110|0000: 0.250  +  0.000i
1001110|0000: 0.250  +  0.000i
0110001|0000: 0.250  +  0.000i
1001001|0000: 0.250  +  0.000i
1010101|0000: 0.250  +  0.000i
0101101|0000: 0.250  +  0.000i
1100011|0000: 0.250  +  0.000i
0011011|0000: 0.250  +  0.000i
0000111|0000: 0.250  +  0.000i
1111111|0000: 0.250  +  0.000i


(<qulacs_core.QuantumState at 0x7f87e8a247f0>, '000')

In [22]:
logical0, anc_meas = run_stabilizer_circuit(gate_sequence + [('H', (i,)) for i in range(7)], num_qubits, num_meas, target_outcomes='000', verbose=True)

ancilla meas: 000
0000000|0000: 0.354  +  0.000i
1111000|0000: 0.354  +  0.000i
0110110|0000: 0.354  +  0.000i
1001110|0000: 0.354  +  0.000i
1010101|0000: 0.354  +  0.000i
0101101|0000: 0.354  +  0.000i
1100011|0000: 0.354  +  0.000i
0011011|0000: 0.354  +  0.000i


In [19]:
computational_states = [''.join(map(str,i))[::-1] for i in itertools.product([0, 1], repeat=num_qubits)]
get_state_dict(state.get_vector(), computational_states)

{'0000000': (0.35355339059327345+0j),
 '1111000': (0.35355339059327345+0j),
 '0110110': (0.35355339059327345+0j),
 '1001110': (0.35355339059327345+0j),
 '1010101': (0.35355339059327345+0j),
 '0101101': (0.35355339059327345+0j),
 '1100011': (0.35355339059327345+0j),
 '0011011': (0.35355339059327345+0j)}

In [26]:
print_state(logical0.get_vector(), computational_states)

0000000|0000: 0.354  +  0.000i
1111000|0000: 0.354  +  0.000i
0110110|0000: 0.354  +  0.000i
1001110|0000: 0.354  +  0.000i
1010101|0000: 0.354  +  0.000i
0101101|0000: 0.354  +  0.000i
1100011|0000: 0.354  +  0.000i
0011011|0000: 0.354  +  0.000i


In [33]:
state = logical0.copy()
circuit = QuantumCircuit(num_qubits)
# error
circuit.add_Z_gate(1)
circuit.add_Z_gate(3)
circuit.add_Z_gate(4)
# correction
circuit.add_Z_gate(6)
circuit.update_quantum_state(state)
print_state(state.get_vector(), computational_states)
get_state_dict(logical0.get_vector(), computational_states) == get_state_dict(state.get_vector(), computational_states)

0000000|0000: 0.354  +  0.000i
1111000|0000: 0.354  +  0.000i
0110110|0000: 0.354  +  0.000i
1001110|0000: 0.354  +  0.000i
1010101|0000: 0.354  +  0.000i
0101101|0000: 0.354  +  0.000i
1100011|0000: 0.354  +  0.000i
0011011|0000: 0.354  +  0.000i


True

In [34]:
state = logical0.copy()
circuit = QuantumCircuit(num_qubits)
# error
circuit.add_Z_gate(0)
circuit.add_Z_gate(1)
circuit.add_Z_gate(5)
circuit.add_Z_gate(6)
# correction
# circuit.add_Z_gate(6)
circuit.update_quantum_state(state)
print_state(state.get_vector(), computational_states)
get_state_dict(logical0.get_vector(), computational_states) == get_state_dict(state.get_vector(), computational_states)

0000000|0000: 0.354  +  0.000i
1111000|0000: 0.354  +  0.000i
0110110|0000: 0.354  +  0.000i
1001110|0000: 0.354  +  0.000i
1010101|0000: 0.354  +  0.000i
0101101|0000: 0.354  +  0.000i
1100011|0000: 0.354  +  0.000i
0011011|0000: 0.354  +  0.000i


True

In [36]:
state = logical0.copy()
circuit = QuantumCircuit(num_qubits)
# error
circuit.add_Z_gate(2)
circuit.add_Z_gate(4)
circuit.add_Z_gate(5)
circuit.add_Z_gate(6)
# correction
circuit.add_Z_gate(2)
circuit.update_quantum_state(state)
print_state(state.get_vector(), computational_states)
get_state_dict(logical0.get_vector(), computational_states) == get_state_dict(state.get_vector(), computational_states)

0000000|0000: 0.354  +  0.000i
1111000|0000: 0.354  +  0.000i
0110110|0000: 0.354  +  0.000i
1001110|0000: 0.354  +  0.000i
1010101|0000: 0.354  +  0.000i
0101101|0000: 0.354  +  0.000i
1100011|0000: 0.354  +  0.000i
0011011|0000: 0.354  +  0.000i


True

In [39]:
state = logical0.copy()
circuit = QuantumCircuit(num_qubits)
# error
circuit.add_Z_gate(0)
circuit.add_Z_gate(1)
circuit.add_Z_gate(2)
circuit.add_Z_gate(3)
circuit.add_Z_gate(4)
circuit.add_Z_gate(5)
circuit.add_Z_gate(6)
# correction
# circuit.add_Z_gate(2)
circuit.update_quantum_state(state)
print_state(state.get_vector(), computational_states)
get_state_dict(logical0.get_vector(), computational_states) == get_state_dict(state.get_vector(), computational_states)

0000000|0000: 0.354  +  0.000i
1111000|0000: 0.354  +  0.000i
0110110|0000: 0.354  +  0.000i
1001110|0000: 0.354  +  0.000i
1010101|0000: 0.354  +  0.000i
0101101|0000: 0.354  +  0.000i
1100011|0000: 0.354  +  0.000i
0011011|0000: 0.354  +  0.000i


True